## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-13-00-41 +0000,wapo,"Trump agencies halt environmental, health proj...",https://www.washingtonpost.com/nation/2025/09/...
1,2025-09-15-13-00-00 +0000,missionlocal,Argali,https://missionlocal.org/2025/09/sf-photos-arg...
2,2025-09-15-12-57-28 +0000,wapo,"Nvidia broke antitrust law, China says on seco...",https://www.washingtonpost.com/world/2025/09/1...
3,2025-09-15-12-52-35 +0000,nyt,"Nvidia Broke Antitrust Law, China Says, as Ten...",https://www.nytimes.com/2025/09/15/business/nv...
4,2025-09-15-12-51-01 +0000,wapo,Utah men arrested after incendiary device is l...,https://www.washingtonpost.com/business/2025/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,31
69,kirk,22
68,charlie,16
75,suspect,9
105,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,96
135,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,86
170,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,83
212,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,80
110,2025-09-15-02-52-08 +0000,nypost,Harvard Law professor spreads debunked rumor a...,https://nypost.com/2025/09/14/us-news/harvard-...,66


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,96,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
2,32,2025-09-15-12-57-28 +0000,wapo,"Nvidia broke antitrust law, China says on seco...",https://www.washingtonpost.com/world/2025/09/1...
185,31,2025-09-14-17-33-46 +0000,nyt,White House Seeks More Supreme Court Security ...,https://www.nytimes.com/2025/09/14/us/politics...
32,29,2025-09-15-11-00-00 +0000,missionlocal,S.F. will be one of the smallest losers in Tru...,https://missionlocal.org/2025/09/sf-smallest-l...
35,28,2025-09-15-10-27-07 +0000,wapo,"Rubio visits Israel, expected to visit Qatar a...",https://www.washingtonpost.com/national-securi...
128,28,2025-09-15-00-43-07 +0000,nypost,‘Lazy’ boozed-up driver busted after driving r...,https://nypost.com/2025/09/14/world-news/booze...
174,27,2025-09-14-18-59-28 +0000,cbc,"Carney announces launch of new housing agency,...",https://www.cbc.ca/news/politics/carney-new-ho...
10,26,2025-09-15-12-12-11 +0000,nypost,Texas Tech student arrested after hurling vile...,https://nypost.com/2025/09/15/us-news/texas-te...
203,26,2025-09-14-15-00-00 +0000,wsj,"Tariffs are rolling into the Treasury, but the...",https://www.wsj.com/economy/capital-gains-tax-...
94,25,2025-09-15-06-35-58 +0000,nypost,UK police say 49-year-old German remains a sus...,https://nypost.com/2025/09/15/world-news/uk-po...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
